## installation and set up

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [ ]:
# setup

from google.colab import userdata
OPENAI_API_KEY=userdata.get('GROUP1_OPENAI_API_KEY')

from google.colab import drive
drive.mount('/content/drive')


from openai import OpenAI

client = OpenAI(
    api_key = OPENAI_API_KEY
)

Mounted at /content/drive


### define assistants

for this tutorial we are going to assume that you have created your assistants in the openai assistant playground (you can also test them out there). Once you've done that, add them to the list below and choose the one(s) you'd like to use.

In [ ]:
# add your assistants here (or just hard code a single assistant as the one you want to use)

group1_assistants = {
    "neanderthal": "asst_vcdGhNv0HpATF4cUv4uilGXz",
    "proto_indo_european": "asst_QATnNPReFpy0OOWLAYoSxyEx",
    "rising_star": "asst_Xvh9jkDpDVRwcTcdsa2LFiXn",
    "sanxingdui": "asst_HQ0ekp0yyBfG8Oj2qUxPtfrp"
}

assistant_id=group1_assistants["sanxingdui"]

## thread cycle

### create

In [ ]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="can you tell me how you can help me, with reference to a quotation from a text you are able to retrieve from? (Please be sure to include at least one quoted passage from the texts you can see, and explain why that might be useful to me)"
)

print(thread)


Thread(id='thread_biV217Wfsx9pjrfIwVqHfHUU', created_at=1712855122, metadata={}, object='thread')


### create the "run" which includes a message and the final prompt (PROMPT A)

### run the thread

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

In [ ]:
import json

print(json.dumps(run.dict(), indent=4))

{
    "id": "run_UYCLLOug7AJaP4Ws7QpGV0JN",
    "assistant_id": "asst_HQ0ekp0yyBfG8Oj2qUxPtfrp",
    "cancelled_at": null,
    "completed_at": null,
    "created_at": 1712855124,
    "expires_at": 1712855724,
    "failed_at": null,
    "file_ids": [
        "file-nNa7pttmuLpJk2tnECz2mckr",
        "file-kMkicos3Xxkx1yWysFotty3B",
        "file-dHABQMCJ9PKqVslmOq7Qw8ZW",
        "file-gvN5M6aM2cg7Q8wJDponOYL4"
    ],
    "instructions": "",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo",
    "object": "thread.run",
    "required_action": null,
    "started_at": null,
    "status": "queued",
    "thread_id": "thread_biV217Wfsx9pjrfIwVqHfHUU",
    "tools": [
        {
            "type": "retrieval"
        }
    ],
    "usage": null,
    "temperature": 1.0
}


### check on the thread

and wait for completion (could take a minute or so).

In [ ]:
import time
import json

def wait_for_completion(client, thread_id, run_id):
    while True:
        run_info = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run_info.status}")
        if run_info.status == "completed":
            print("Run is complete.")
            break
        else:
            # Wait for a short period before checking again.
            print(f"not done yet, status is {run_info.status}")
            time.sleep(5)  # Adjust sleep time as needed.

    return run_info

run_info = wait_for_completion(client, thread.id, run.id)

print(json.dumps(run_info.dict(), indent=4))

Current run status: in_progress
not done yet, status is in_progress
Current run status: in_progress
not done yet, status is in_progress
Current run status: in_progress
not done yet, status is in_progress
Current run status: completed
Run is complete.
{
    "id": "run_UYCLLOug7AJaP4Ws7QpGV0JN",
    "assistant_id": "asst_HQ0ekp0yyBfG8Oj2qUxPtfrp",
    "cancelled_at": null,
    "completed_at": 1712855142,
    "created_at": 1712855124,
    "expires_at": null,
    "failed_at": null,
    "file_ids": [
        "file-nNa7pttmuLpJk2tnECz2mckr",
        "file-kMkicos3Xxkx1yWysFotty3B",
        "file-dHABQMCJ9PKqVslmOq7Qw8ZW",
        "file-gvN5M6aM2cg7Q8wJDponOYL4"
    ],
    "instructions": "",
    "last_error": null,
    "metadata": {},
    "model": "gpt-4-turbo",
    "object": "thread.run",
    "required_action": null,
    "started_at": 1712855124,
    "status": "completed",
    "thread_id": "thread_biV217Wfsx9pjrfIwVqHfHUU",
    "tools": [
        {
            "type": "retrieval"
        }


### retrieve the output and print

plus new logic that sorts through ALL messages and only returns the last-produced JSON string.

In [ ]:
import json
import textwrap

messages = client.beta.threads.messages.list(thread_id=thread.id)

# FOR ALL MESSAGES UNCOMMENT THIS

# for message in messages.data:
#     message_info = client.beta.threads.messages.retrieve(
#         thread_id=thread.id,
#         message_id=message.id
#     )
#     print(json.dumps(message_info.dict(), indent=4))
#     message_text = message_info.content[0].text.value
#     print(textwrap.fill(message_text, width=50))

# JUST MOST RECENT MESSAGE

message_info = client.beta.threads.messages.retrieve(
    thread_id=thread.id,
    message_id=messages.data[0].id
)

print(json.dumps(message_info.dict(), indent=4))
message_text = message_info.content[0].text.value
print(textwrap.fill(message_text, width=50))

{
    "id": "msg_m1i9tGQCXsoIqo5T6DDisz8k",
    "assistant_id": "asst_HQ0ekp0yyBfG8Oj2qUxPtfrp",
    "completed_at": null,
    "content": [
        {
            "text": {
                "annotations": [
                    {
                        "end_index": 564,
                        "file_citation": {
                            "file_id": "file-gvN5M6aM2cg7Q8wJDponOYL4",
                            "quote": ""
                        },
                        "start_index": 553,
                        "text": "\u301011\u2020source\u3011",
                        "type": "file_citation"
                    }
                ],
                "value": "I can assist you by providing detailed information from the texts I can fetch and analyze, which could be helpful in understanding specific topics or facilitating learning and research. For instance, consider this passage from an article about the Sanxingdui Bronze Age relics:\n\n\"[The Sanxingdui Museum] contains the ruins of

In [ ]:
def process_citations(client, message_content):
    # Copy the original message content for this run
    modified_content = message_content.value[:]
    annotations = message_content.annotations if hasattr(message_content, 'annotations') else []
    citations = []

    for index, annotation in enumerate(annotations):
        # Replace the text with a footnote marker
        modified_content = modified_content.replace(annotation.text, f' [{index}]')

        # Process file citations and paths
        if (file_citation := getattr(annotation, 'file_citation', None)):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f'[{index}] {file_citation.quote} from {cited_file.filename}')
        elif (file_path := getattr(annotation, 'file_path', None)):
            cited_file = client.files.retrieve(file_path.file_id)
            citations.append(f'[{index}] Click <here> to download {cited_file.filename}')
            # Placeholder for actual file download link or method

    # Append gathered citations, ensuring they start on a new line
    if citations:
        modified_content += '\n\n' + '\n'.join(citations)  # Ensure two newlines before starting the citations

    # Wrap the modified content, including citations, with 50 character line width
    wrapped_content = textwrap.fill(modified_content, width=50)

    return wrapped_content

# Use the function to process and wrap your content, adjust function parameters as needed
wrapped_and_processed_content = process_citations(client, original_message_content)

# Print the wrapped and processed content with correct newline before citations
print(wrapped_and_processed_content)

I can assist you by providing detailed information
from the texts I can fetch and analyze, which
could be helpful in understanding specific topics
or facilitating learning and research. For
instance, consider this passage from an article
about the Sanxingdui Bronze Age relics:  "[The
Sanxingdui Museum] contains the ruins of an
ancient city made up of residences, sacrificial
pits and tombs enclosed by high dirt walls.
Archaeologists say the city was established some
4,800 to 2,800 years ago, until it was abandoned
around 800 BC for unknown reasons" [0].  This
quote suggests a detailed and rich view into a
once-thriving civilization, capturing the
imagination and possibly answering questions
related to ancient cultures, archaeological
methods, or the use of material culture to
understand the past. Such insights could be
helpful for educational purposes, further
research, or personal interest, demonstrating just
one of the many ways this service can be useful to
you depending on your need